In [1053]:
# Path ORAM 

from collections import defaultdict, Counter
import random

class Node(object): 
	def __init__(self, ary, idx, left, right): 
		self.val = ary
		self.idx = idx
		self.left = left 
		self.right = right
		
	def __repr__(self):
		return f"({self.idx},{self.val},{self.left},{self.right})"


# constructing a simple binary tree
# actually each of the int in the arrary should be a pointer to a block of size B
#    0    l0
#   /\
#  1  2   l1
#  /\ /\
# 3 4 5 6 l2

# n3 = Node([("", 0),("", 0),("", 0),("", 0)], 3, None, None) 
# n4 = Node([("", 0),("", 0),("", 0),("", 0)], 4, None, None)
# n5 = Node([("", 0),("", 0),("", 0),("", 0)], 5, None, None)
# n6 = Node([("", 0),("", 0),("", 0),("", 0)], 6, None, None)

# n1 = Node([("", 0),("", 0),("", 0),("", 0)], 1, n3, n4)
# n2 = Node([("", 0),("", 0),("", 0),("", 0)], 2, n5, n6)

# n0 = Node([("", 0),("", 0),("", 0),("", 0)], 0, n1, n2)

n3 = Node([], 3, None, None) 
n4 = Node([], 4, None, None)
n5 = Node([], 5, None, None)
n6 = Node([], 6, None, None)

n1 = Node([], 1, n3, n4)
n2 = Node([], 2, n5, n6)

n0 = Node([], 0, n1, n2)

print(n0)

(0,[],(1,[],(3,[],None,None),(4,[],None,None)),(2,[],(5,[],None,None),(6,[],None,None)))


In [1054]:
def getHeight(root):
	'''
	Assumption: the given tree is a binary tree 
	Input: the root node of a tree 
	Output: the height of the tree 
	'''
	if not root: 
		return 0 

	leftHeight = getHeight(root.left)
	rightHeight = getHeight(root.right) 

	return max(leftHeight, rightHeight) + 1 

In [1055]:
LEVELS = getHeight(n0)
print(LEVELS)

3


In [1056]:
# Global set up 
N = 28 
Z = 4 # Z numbers of blocks within each bucket
global STASH 
STASH = []
random.seed(11)
# position = defaultdict(int, {k:random.randrange((pow(2, LEVELS - 1) - 1), (pow(2, LEVELS) - 1)) for k in range(N)})
position = defaultdict(int, {k:random.randrange(0, (pow(2, LEVELS) - 1)) for k in range(1,N+1)})

# The hard coded tree does not match the position map 
sorted_pos = defaultdict(list)
# {k : v if k in position else 0 for k,v in position.items()}

for key,val in position.items():
    sorted_pos[val].append(key)
    

In [1057]:
print(position)
print(sorted_pos)

defaultdict(<class 'int'>, {1: 3, 2: 6, 3: 4, 4: 6, 5: 6, 6: 3, 7: 3, 8: 4, 9: 6, 10: 4, 11: 1, 12: 1, 13: 6, 14: 4, 15: 3, 16: 5, 17: 4, 18: 6, 19: 1, 20: 0, 21: 3, 22: 2, 23: 1, 24: 0, 25: 4, 26: 6, 27: 5, 28: 5})
defaultdict(<class 'list'>, {3: [1, 6, 7, 15, 21], 6: [2, 4, 5, 9, 13, 18, 26], 4: [3, 8, 10, 14, 17, 25], 1: [11, 12, 19, 23], 5: [16, 27, 28], 0: [20, 24], 2: [22]})


In [1058]:
def concat_STASH(anotherL):
    for i in anotherL:
        STASH.append(i)
    return None 

In [1059]:

def initialize_tree(root):
    if root is None:
        return None 

    data_ = [("", x) for x in sorted_pos[root.idx]]
    data = data_[:Z]
    concat_STASH(data_[Z:])
    print("STASH is {}".format(STASH))
    
    root.val = data 
    initialize_tree(root.left)
    initialize_tree(root.right)
    return None

initialize_tree(n0)
n0

STASH is []
STASH is []
STASH is [('', 21)]
STASH is [('', 21), ('', 17), ('', 25)]
STASH is [('', 21), ('', 17), ('', 25)]
STASH is [('', 21), ('', 17), ('', 25)]
STASH is [('', 21), ('', 17), ('', 25), ('', 13), ('', 18), ('', 26)]


(0,[('', 20), ('', 24)],(1,[('', 11), ('', 12), ('', 19), ('', 23)],(3,[('', 1), ('', 6), ('', 7), ('', 15)],None,None),(4,[('', 3), ('', 8), ('', 10), ('', 14)],None,None)),(2,[('', 22)],(5,[('', 16), ('', 27), ('', 28)],None,None),(6,[('', 2), ('', 4), ('', 5), ('', 9)],None,None)))

In [1060]:
def getPath(root, NodeId): 
	'''
	Input: root(root) node, and leaf node(NodeId)
	Output: all nodes along the path from leaf node to root node. 
	'''
	if not root:
		return []
	
	if root.idx == NodeId:
		return [(root.idx, root.val)] 

	leftPath = getPath(root.left, NodeId) 
	rightPath = getPath(root.right, NodeId)
 
	if leftPath:
		return leftPath +  [(root.idx, root.val)]
	if rightPath: 
		return rightPath + [(root.idx, root.val)]
	return []

In [1061]:
print(getPath(n0, 6))

[(6, [('', 2), ('', 4), ('', 5), ('', 9)]), (2, [('', 22)]), (0, [('', 20), ('', 24)])]


In [1062]:
# inspection 
def getDataInBlock(root, blockId):
  ''' 
  Get the data associated with a block ID in a given tree
  Input: root of a tree and a block ID
  Output: the data associated with the block ID 
  '''
  if root is not None:
    # it should be iterator here rather than fix length traversal 
    for elem in iter(root.val):
      if elem[1] == blockId:
        return elem[0]
    # for i in range(Z):
    #   if(root.val): #and len(root.val) == Z:
    #     if(root.val[i][1] == blockId):
    #       return root.val[i][0]

    lres = getDataInBlock(root.left, blockId)
    if lres is not None: 
      return lres 

    rres = getDataInBlock(root.right, blockId)
    if rres is not None:
      return rres

getDataInBlock(n0,16)

''

In [1063]:
def getNodeAtLevel(root, leafidx, level):
  '''
  Input: the root of a tree, the leaf node index, a specific level
  Output: the node along the path from a given leaf node to root node at the given level 
  '''
  path = getPath(root, leafidx)
  path_len = len(path)
  if level > path_len - 1:
    return None
  return path[path_len-level-1]


In [1064]:
def ReadBucket(root, leaf):
    '''
    read all of the (block, data) pair along the path from a leaf node to the root node 
    Input: the root of a tree, a leaf node, and a 

    '''
    # STASH = []
    for l in range(LEVELS):
        bucket = getNodeAtLevel(root, leaf, l)
        if bucket is not None:
            bucket = bucket[1]
            print("bucket is {}".format(bucket))
            for elem in bucket: 
                STASH.append(elem)
    return STASH
    
# ReadBucket(n0, 5)


In [1065]:
def getCandidateBlocksHelper(root, leaf, blockID, level): 
	'''Output: (node.idx, node.val)'''
	# print("--------------------------------------------------------------------------------------------inspection function")
	# print("leaf is {} blockID is {} level is{}".format(leaf, blockID, level))
	# print("current blockid {} maps to leaf {} at level {}".format(blockID,position[blockID], level))
	lhs = getNodeAtLevel(root, leaf, level)
	# print("left hand side is ")
	# print(lhs)
	# print("right hand side is ")	
	rhs = getNodeAtLevel(root, position[blockID], level)
	# print(rhs)
 
	if lhs is not None and rhs is not None:
		if lhs == rhs:
			return (getDataInBlock(root, blockID), blockID)
		else:
			return None 
	return None

In [1066]:
def testing(leaf, blockid):
    for i in range(3):
        print("Testing at Level {}".format(i))
        print(getCandidateBlocksHelper(n0, leaf, blockid, i))        

In [1067]:
def testall(): 
    for leaf,bucket in sorted_pos.items():
        print("---------------{}-------------".format(leaf))
        for bid in bucket:
            print("=========={}===========".format(bid))
            testing(leaf, bid)

# testall()


In [1068]:
# {4: [1, 5, 17, 28], 0: [2, 6, 16, 20, 24], 3: [3, 4, 8, 10, 18, 25], 1: [7, 15, 21, 27], 6: [9, 11, 14, 26], 2: [12, 19, 23], 5: [13, 22]}
# 4 3 1 
# print(getCandidateBlocksHelper(n0, 3, 2, 0))
print(getCandidateBlocksHelper(n0, 3, 2, 1))
# print(getCandidateBlocksHelper(n0, 3, 4, 2))

None


In [1069]:
def getCandidateBlocks(root, leaf, level, stsh): 
    acc = []
    # print("from leaf {}".format(leaf))
    for elem in stsh:
        # print("idb is {}".format(idb))
        blocks = getCandidateBlocksHelper(root, leaf, elem[1], level)
        # print("blocks {} on the same path as leaf {} are blocks {}".format(idb, leaf, blocks))
        acc.append(blocks)
    res = [x for x in acc if x is not None]
    return res

In [1070]:
def writeBackNodes(root, index, currlevel, tgtlevel, data):
	if root is None:
		return None

	if root.idx == index and currlevel == tgtlevel:
		retVal = root.val 
		root.val = data 
		return retVal

	nextlevel = currlevel + 1

	leftTree = writeBackNodes(root.left, index, nextlevel, tgtlevel, data)
	if leftTree: 
		return leftTree
  
	rightTree = writeBackNodes(root.right, index, nextlevel, tgtlevel, data)
	if rightTree:
		return rightTree

	return None  

In [1071]:
def update_STASH(blockID, dataN, stsh):
    blockIndex = -1
    for (idx, pair) in enumerate(stsh):
        temp = list(pair)
        if temp[1] == blockID:
            blockIndex = idx 
    stsh[blockIndex] = (dataN, blockID)

In [1072]:
def pop_STASH(stsh, items):
    '''
    Input: STASH, and list of pairs to be popped 
    Output: updated STASH 
    '''
    # updatedSTASH = [x for x in stsh if x not in items]
    for elem in stsh:
        if elem in items:
            stsh.remove(elem)
    return stsh

pop_STASH([('d',1),('a', 2),('t',3)],[('d',1)])

[('a', 2), ('t', 3)]

In [1073]:
def print_tree_status(status):
    print(status +" tree is ")
    print(n0)

In [1074]:
from turtle import update


def access(root, opCode, blockId, dataNew): 
	leafIdx = position[blockId]
	print("Node blockID {} maps to leaf {}".format(blockId, leafIdx))
	position[blockId] = random.randrange((pow(2, LEVELS - 1) - 1), (pow(2, LEVELS) - 1))
	print("new leaf is {}".format(position[blockId]) )
	
	STASH = ReadBucket(root, leafIdx)
	print("root is {}, leaf is {}".format(root, leafIdx))
	print("Current STASH is ")
	print(STASH)

	# read block a from STASH
	dataOld = getDataInBlock(root, blockId)
	print("old data is ")
	print(dataOld)
	

	if opCode == "wr": 
		# (check if the blockId is valid): do we assume a perfect binary tree? 
		# STASH = [item for item in STASH if item not in [STASH[blockId]]]
		updateNode = getNodeAtLevel(root, leafIdx, LEVELS - 1)
		update_STASH(blockId, dataNew, STASH)
		print("updated STASH is {}".format(STASH))
		print("========== node to be written is  ==========")
		print(updateNode)
 
	# write path 
	for l in reversed(range(LEVELS)):
		print("_*_*&_*&_*&-_*&qw#_*re&_*-&!@$*_&!@-#.---{}---&_*&-_*&qw#_*re&_*-&!@$*_&!@-".format(l))		
		
		if len(STASH) is not 0:
			candidateBlocks = getCandidateBlocks(root, leafIdx, l, STASH)
			if (len(candidateBlocks) >= Z):
				writeBackSize = Z
			else:
				writeBackSize = len(candidateBlocks)
			writeBackBlocks = candidateBlocks[:writeBackSize]
		else:
			candidateBlocks = []
			writeBackSize = 0

		print("CandidateBlock at level {}".format(l))
		print(candidateBlocks)
		print("writeBacBlock at level {}".format(l))
		print(writeBackBlocks)
		poplist = candidateBlocks[writeBackSize:]
		print("poplist is {}".format(poplist))
		updatedSTASH = pop_STASH(STASH, poplist)
		print("Popped STASH is {}".format(updatedSTASH))
		# print_tree_status("tree &&&&&&&&&& before poplist is")
		print_tree_status("tree before write back is")
		writeBackNodes(root, leafIdx, 0, l, writeBackBlocks)
	return dataOld
	

<>:34: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:34: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/r1/qkm3tcjx2ys2lcd0ycgbh3h80000gn/T/ipykernel_1077/2165326832.py:34: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if len(STASH) is not 0:


In [1075]:
print_tree_status("initialized")
oldD = access(n0, "wr", 16, "x")
print_tree_status("after first write")

initialized tree is 
(0,[('', 20), ('', 24)],(1,[('', 11), ('', 12), ('', 19), ('', 23)],(3,[('', 1), ('', 6), ('', 7), ('', 15)],None,None),(4,[('', 3), ('', 8), ('', 10), ('', 14)],None,None)),(2,[('', 22)],(5,[('', 16), ('', 27), ('', 28)],None,None),(6,[('', 2), ('', 4), ('', 5), ('', 9)],None,None)))
Node blockID 16 maps to leaf 5
new leaf is 3
bucket is [('', 20), ('', 24)]
bucket is [('', 22)]
bucket is [('', 16), ('', 27), ('', 28)]
root is (0,[('', 20), ('', 24)],(1,[('', 11), ('', 12), ('', 19), ('', 23)],(3,[('', 1), ('', 6), ('', 7), ('', 15)],None,None),(4,[('', 3), ('', 8), ('', 10), ('', 14)],None,None)),(2,[('', 22)],(5,[('', 16), ('', 27), ('', 28)],None,None),(6,[('', 2), ('', 4), ('', 5), ('', 9)],None,None))), leaf is 5
Current STASH is 
[('', 21), ('', 17), ('', 25), ('', 13), ('', 18), ('', 26), ('', 20), ('', 24), ('', 22), ('', 16), ('', 27), ('', 28)]
old data is 

updated STASH is [('', 21), ('', 17), ('', 25), ('', 13), ('', 18), ('', 26), ('', 20), ('', 24),